# Modeling and Simulation in Python

Chapter 4: Predict

Copyright 2017 Allen Downey

License: [Creative Commons Attribution 4.0 International](https://creativecommons.org/licenses/by/4.0)


In [1]:
# If you want the figures to appear in the notebook, 
# and you want to interact with them, use
# %matplotlib notebook

# If you want the figures to appear in the notebook, 
# and you don't want to interact with them, use
# %matplotlib inline

# If you want the figures to appear in separate windows, use
# %matplotlib qt5

# To switch from one to another, you have to select Kernel->Restart

%matplotlib qt5

from modsim import *

### Functions from the previous chapter

In [2]:
def plot_estimates(table):
    """Plot world population estimates.
    
    table: DataFrame with columns `un` and `census`
    """
    un = table.un / 1e9
    census = table.census / 1e9
    
    plot(census, ':', color='darkblue', label='US Census')
    plot(un, '--', color='green', label='UN DESA')
    
    decorate(xlabel='Year',
             ylabel='World population (billion)')

In [3]:
def plot_results(system):
    """Plot the estimates and the model.
    
    system: System object with `results`
    """
    newfig()
    plot_estimates(table2)
    plot(system.results, '--', color='gray', label='model')
    decorate(xlabel='Year', 
             ylabel='World population (billion)')

In [4]:
def run_simulation(system, update_func):
    """Run a model.
    
    Adds TimeSeries to `system` as `results`.

    system: System object
    update_func: function that computes the population next year
    """
    results = Series([])
    results[system.t0] = system.p0
    for t in linrange(system.t0, system.t_end):
        results[t+1] = update_func(results[t], t, system)
    system.results = results

### Reading the data

In [5]:
# The data directory contains a downloaded copy of
# https://en.wikipedia.org/wiki/World_population_estimates

from pandas import read_html
filename = 'data/World_population_estimates.html'
tables = read_html(filename, header=0, index_col=0, decimal='M')

In [6]:
table2 = tables[2]

In [7]:
table2.columns = ['census', 'prb', 'un', 'maddison', 
                  'hyde', 'tanton', 'biraben', 'mj', 
                  'thomlinson', 'durand', 'clark']

In [8]:
newfig()
plot_estimates(table2)

### Running the quadratic model

Here's the update function for the quadratic growth model with parameters `alpha` and `beta`.

In [9]:
def update_func2(pop, t, system):
    """Update population based on a quadratic model.
    
    pop: current population in billions
    t: what year it is
    system: system object with model parameters
    """
    net_growth = system.alpha * pop + system.beta * pop**2
    return pop + net_growth

Select the estimates generated by the U.S. Census, and convert to billions.

In [10]:
census = table2.census / 1e9

Extract the starting time and population.

In [18]:
t0 = census.index[0]
p0 = census[t0]
t_end = census.index[-1]

Initialize the system object.

In [12]:
variables = System(t0=t0, 
                t_end=t_end,
                p0=p0,
                alpha=0.025,
                beta=-0.0018)

variables

,value
t0,1950.000000
t_end,2015.000000
p0,2.557629
alpha,0.025000
beta,-0.001800


Run the model and plot results.

In [13]:
run_simulation(variables, update_func2)
plot_results(variables)
decorate(title='Quadratic model')

### Generating projections

To generate projections, all we have to do is change `t_end`

In [14]:
variables.t_end = 2250
run_simulation(variables, update_func2)
plot_results(variables)
decorate(title='World population projection')
savefig('chap04-fig01.pdf')

Saving figure to file chap04-fig01.pdf


The population in the model converges on the equilibrium population, `-alpha/beta`

In [15]:
variables.results[variables.t_end]

13.856665141368708

In [16]:
-variables.alpha / variables.beta

13.888888888888889

**Exercise:**  What happens if we start with an initial population above the carrying capacity, like 20 billion?  The the model with initial populations between 1 and 20 billion, and plot the results on the same axes.

In [22]:
variables.p0 = 20
run_simulation(variables, update_func2)
plot_results(variables)
variables.p0 = p0
run_simulation(variables, update_func2)
plot(variables.results)
decorate(title='World Population Overload')

### Comparing projections

We can compare the projection from our model with projections produced by people who know what they are doing.

In [25]:
table3 = tables[3]
table3.head()

,census,prb,un
Year,,,
2016,7.334772e+09,NaN,7.432663e+09
2017,7.412779e+09,NaN,NaN
2018,7.490428e+09,NaN,NaN
2019,7.567403e+09,NaN,NaN
2020,7.643402e+09,NaN,7.758157e+09


`NaN` is a special value that represents missing data, in this case because some agencies did not publish projections for some years.

In [24]:
table3.columns = ['census', 'prb', 'un']

This function plots projections from the UN DESA and U.S. Census.  It uses `dropna` to remove the `NaN` values from each series before plotting it.

In [26]:
def plot_projections(table):
    """Plot world population projections.
    
    table: DataFrame with columns 'un' and 'census'
    """
    census = table.census / 1e9
    un = table.un / 1e9
    
    plot(census.dropna(), ':', color='darkblue', label='US Census')
    plot(un.dropna(), '--', color='green', label='UN DESA')

Run the model until 2100, which is as far as the other projections go.

In [28]:
variables.p0 = census[t0]
variables.t_end = 2100
run_simulation(variables, update_func2)

Plot the results.

In [29]:
plot_results(variables)
plot_projections(table3)
decorate(title='World population projections')
savefig('chap04-fig02.pdf')

Saving figure to file chap04-fig02.pdf


People who know what they are doing expect the growth rate to decline more sharply than our model projects.

**Exercise:**  Suppose there are two banks across the street from each other, The First Geometric Bank (FGB) and Exponential Savings and Loan (ESL).  They offer the same interest rate on checking accounts, 3%, but at FGB, they compute and pay interest at the end of each year, and at ESL they compound interest continuously.

If you deposit $p_0$ dollars at FGB at the beginning of Year 0, the balanace of your account at the end of Year $n$ is

$ x_n = p_0 (1 + \alpha)^n $

where $\alpha = 0.03$.  At ESL, your balance at any time $t$ would be

$ x(t) = p_0 \exp(\alpha t) $

If you deposit \$1000 at each back at the beginning of Year 0, how much would you have in each account after 10 years?

Is there an interest rate FGB could pay so that your balance at the end of each year would be the same at both banks?  What is it?

Hint: `modsim` provides a function called `exp`, which is a wrapper for the NumPy function `exp`.

In [27]:
def input_money_FGB(t):
    return 1000 * (1 + 0.03)**t

In [9]:
input_money_FGB()

1343.9163793441223

In [28]:
def input_money_ESL(t):
    return 1000 * exp(0.03 * t)

In [11]:
input_money_ESL()

1349.8588075760031

In [12]:
e = input_money_ESL()
x = log(e / 1000) / log(0.03 + 1)
print(x, 'years')

10.1492610407 years


In [13]:
input_money_FGB(t=x)

1349.8588075760031

In [14]:
input_money_ESL()

1349.8588075760031

**Exercise:** Suppose a new bank opens called the Polynomial Credit Union (PCU).  In order to compete with First Geometric Bank and Exponential Savings and Loan, PCU offers a parabolic savings account where the balance is a polynomial function of time:

$ x(t) = p_0 + \beta_1 t + \beta_2 t^2 $

As a special deal, they offer an account with $\beta_1 = 30$ and $\beta_2 = 0.5$, with those parameters guaranteed for life.

Suppose you deposit \$1000 at all three banks at the beginning of Year 0.  How much would you have in each account at the end of Year 10?  How about Year 20?  And Year 100?

In [35]:
def input_money_PCU(t):
    return 1000 + 30 * t + 0.5 *t**2

In [36]:
def run_simulation(t):
    x = input_money_FGB(t)
    y = input_money_ESL(t)
    z = input_money_PCU(t)
    print('FGB=', x)
    print('ESL=', y)
    print('PCU=', z)

In [37]:
run_simulation(10)

FGB= 1343.9163793441223
ESL= 1349.85880758
PCU= 1350.0


In [38]:
run_simulation(20)

FGB= 1806.111234669415
ESL= 1822.11880039
PCU= 1800.0


In [39]:
run_simulation(100)

FGB= 19218.6319808563
ESL= 20085.5369232
PCU= 9000.0
